In [ ]:
# Import modules and add library to path
import sys
import os

os.environ["CUDA_VISIBLE_DEVICES"] = ""  # Change to os.environ["CUDA_VISIBLE_DEVICES"] = "" if you dont have access to a GPU
import pickle

import numpy as np
import pandas as pd
# Initialize plotly figures
from plotly.offline import init_notebook_mode

# DeepMReye imports
from deepmreye import analyse, preprocess, train
from deepmreye.util import data_generator, model_opts

init_notebook_mode(connected=True)

# Make sure the output width is adjusted for better export as HTML
from IPython.core.display import HTML, display

display(HTML("<style>.container { width:70% !important; }</style>"))
display(HTML("<style>.output_result { max-width:70% !important; }</style>"))

# Autoreload modules
%load_ext autoreload
%autoreload 2

# Preprocessing

In [ ]:
# Define paths to functional data
experiment_folder = 'F:\\Data Fusion Project\\data\\deepmreye'
functional_data = os.path.join(experiment_folder, "functional_data")
processed_data = os.path.join(experiment_folder, "processed_data/")
model_weights = os.path.join(experiment_folder, "model_weights", "datasets_1to6.h5")

# Get participants from functional folder
# (if needed, remove single participants with participants.remove('participant01') or recreate participants list)
participants = os.listdir(functional_data)

In [ ]:
# Preload masks to save time within participant loop
(
    eyemask_small,
    eyemask_big,
    dme_template,
    mask,
    x_edges,
    y_edges,
    z_edges,
) = preprocess.get_masks()

# Loop across participants and extract eye mask
for participant in participants:

    if participant.startswith("S"):

        print(f"Running participant {participant}")
        participant_folder = os.path.join(functional_data, participant)

        for run in os.listdir(participant_folder):

            if run.startswith("fwhm0"):

                # Filepath to functional
                fp_func = os.path.join(participant_folder, run)

                preprocess.run_participant(
                    fp_func,
                    dme_template,
                    eyemask_big,
                    eyemask_small,
                    x_edges,
                    y_edges,
                    z_edges,
                    transforms=['Affine', 'Affine', 'SyNAggro']
                )

In [ ]:
# Combine processed masks with labels
for participant in participants:

    if participant.startswith("S"):

        print(f"Running participant {participant}")
        participant_folder = os.path.join(functional_data, participant)

        participant_data, participant_labels, participant_ids = [], [], []

        for run_idx, run in enumerate(os.listdir(participant_folder)):

            if not run.endswith(".p"):
                continue

            # Load mask and normalize it
            this_mask = os.path.join(participant_folder, run)
            this_mask = pickle.load(open(this_mask, "rb"))
            this_mask = preprocess.normalize_img(this_mask)

            # If experiment has no labels use dummy labels
            this_label = np.zeros(
                (this_mask.shape[3], 10, 2)
            )  # 10 is the number of subTRs used in the pretrained weights, 2 is XY

            # Check if each functional image has a corresponding label.
            # Note that mask has time as third dimension
            if this_mask.shape[3] != this_label.shape[0]:
                print(
                    f"WARNING --- Skipping Subject {participant} Run {run_idx} "
                    f"--- Wrong alignment (Mask {this_mask.shape} - Label {this_label.shape})."
                )

                continue

            # Store across runs
            participant_data.append(this_mask)
            participant_labels.append(this_label)
            participant_ids.append(([participant] * this_label.shape[0],
                                    [run_idx] * this_label.shape[0]))

        # Save participant file
        preprocess.save_data(
            f"{participant}no_label",
            participant_data,
            participant_labels,
            participant_ids,
            processed_data,
            center_labels=False,
        )

# Load & visualize input

In [ ]:
# Define paths to example dataset
datasets = [
    processed_data + p for p in os.listdir(processed_data) if "no_label" in p
]

# Load data from one participant to showcase input/output
X, y = data_generator.get_all_subject_data(datasets[0])
print(f"Input: {X.shape}, Output: {y.shape}")

In [ ]:
fig = analyse.visualise_input_data(X,
                                   y,
                                   bg_color="rgb(255,255,255)",
                                   ylim=[-11, 11])
fig.show()

# Load model weights

In [ ]:
opts = model_opts.get_opts()
test_participants = [
    processed_data + p for p in os.listdir(processed_data) if "no_label" in p
]
generators = data_generator.create_generators(test_participants,
                                              test_participants)
generators = (*generators, test_participants, test_participants
              )  # Add participant list

In [ ]:
# Get untrained model and load with trained weights
(model, model_inference) = train.train_model(dataset="fmri_data_fusion",
                                             generators=generators,
                                             opts=opts,
                                             return_untrained=True)
model_inference.load_weights(model_weights)

# Model evaluation

In [ ]:
(evaluation, scores) = train.evaluate_model(
    dataset="fmri_data_fusion",
    model=model_inference,
    generators=generators,
    save=False,
    model_path=experiment_folder,
    model_description="",
    verbose=2,
    percentile_cut=80,
)

In [ ]:
fig = analyse.visualise_predictions_slider(
    evaluation,
    scores,
    color="rgb(0, 150, 175)",
    bg_color="rgb(255,255,255)",
    ylim=[-11, 11],
)
fig.show()

# Visualize EVALUATION

In [ ]:
import uuid
import tkinter as tk
from tkinter import ttk

def j_tree(tree, parent, dic):
    for key in sorted(dic.keys()):
        uid = uuid.uuid4()
        if isinstance(dic[key], dict):
            tree.insert(parent, 'end', uid, text=key)
            j_tree(tree, uid, dic[key])
        elif isinstance(dic[key], tuple):
            tree.insert(parent, 'end', uid, text=str(key) + '()')
            j_tree(tree, uid,
                   dict([(i, x) for i, x in enumerate(dic[key])]))
        elif isinstance(dic[key], list):
            tree.insert(parent, 'end', uid, text=str(key) + '[]')
            j_tree(tree, uid,
                   dict([(i, x) for i, x in enumerate(dic[key])]))
        else:
            value = dic[key]
            if isinstance(value, str):
                value = value.replace(' ', '_')
            tree.insert(parent, 'end', uid, text=key, value=value)

def tk_tree_view(data):
    # Setup the root UI
    root = tk.Tk()
    root.title("tk_tree_view")
    root.columnconfigure(0, weight=1)
    root.rowconfigure(0, weight=1)

    # Setup the Frames
    tree_frame = ttk.Frame(root, padding="3")
    tree_frame.grid(row=0, column=0, sticky=tk.NSEW)

    # Setup the Tree
    tree = ttk.Treeview(tree_frame, columns=('Values'))
    tree.column('Values', width=100, anchor='center')
    tree.heading('Values', text='Values')
    j_tree(tree, '', data)
    tree.pack(fill=tk.BOTH, expand=1)

    # Limit windows minimum dimensions
    root.update_idletasks()
    root.minsize(root.winfo_reqwidth(), root.winfo_reqheight())
    root.mainloop()

In [ ]:
tk_tree_view(evaluation)

# Edit output

In [ ]:
gaze = {}
participants = os.listdir(functional_data)
p = 0

for key, item in evaluation.items():
    gaze[participants[p]] = np.reshape(item['pred_y'],(item['pred_y'].shape[0] * item['pred_y'].shape[1], -1))
    p = p+1

In [ ]:
from scipy.io import savemat

scipy.io.savemat('gaze.mat', gaze)